In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 9 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Des EDP aux EDO
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Des EDP aux EDO

In [ ]:
from dataclasses import dataclass
import numpy as np
from scipy.integrate import solve_ivp
from scipy.sparse import diags
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

## Equation de la chaleur

On considère l'équation de la chaleur dans le cas 1D :

$$
\left\{
\begin{aligned}
& \partial_t u(x,t) - D \, \partial_{xx} u(x,t) = 0 \;\; \text{pour} \; -L < x < L\\
& \partial_{x} u(-L,t) = \partial_{x} u(L,t) = 0,\\
& u(x,t=t_0)  =  u_0(x) = \frac{1}{2 \sqrt{\pi D t_0}} \exp \Bigg(-\frac{x^2}{\sqrt{4Dt_0}} \Bigg),
\end{aligned}
\right.
$$

où la donnée initiale est donc la solution élémentaire de l'équation de la chaleur au temps $t_0$ et 
avec des conditions aux limites de type Neumann homogène.

**Discrétisation spatiale**

L'intervalle $[-L,L]$ est discrétisé uniformément en $N+1$ points $(x_i)_{0\leq i \leq N}$:

$$
x_0=-L,\qquad x_{i+1} = x_i + \Delta x,\quad \Delta x = 2L/N.
$$

On note $U(t) \in R^{N+1}$ le vecteur dont les composantes $U_{i}(t)$ sont tels que $U_{i}(t) = u(t,x_i)$ pout tout $i$ dans $\{0,1,\ldots,N\}$. 

Afin de construire une approximation de la solution, nous nous appuierons sur une discrétisation par différences finies de l'opérateur de Laplace : 

$$
\partial^{2}_{xx} u (t,x_i) \approx \displaystyle\frac{1}{(\Delta x)^2} 
( u(t,x_i + \Delta x) - 2u(t,x_i) + u(t,x_i - \Delta x))
= \frac{1}{(\Delta x)^2} 
( U_{i+1}(t) - 2U_{i}(t) + U_{i-1}(t)) \quad \forall i = 0, \ldots, N.
$$

Les equations pour $i=0$ and $i=N$ implique $U_{-1}$ and $U_{N+1}$ dont les valeurs seront déduites aux conditions aux limites. Au niveau discret, en prenant une différence centrée pour approximer la dérivée du premier ordre, les conditions aux limites homogènes de Neumann reviennent à imposer :

$$
\frac{u(t,x_{0}-\Delta x) - u(t,x_{0}+\Delta x)}{2\Delta x} = 0 = \frac{u(t,x_{N}-\Delta x) - u(t,x_{N}+\Delta x)}{2\Delta x} \quad~\forall~t\geq 0,
$$

soit : 

$$
U_{-1}(t) = U_{1}(t) \text{ and } U_{N+1}(t) = U_{N-1}(t)\quad~\forall~t\geq 0.
$$

En réarrangeant les termes, on obtient une approximation $\overline U$ of $U$ en résolvant le système d'équations différentiels ordinaires suivant : 

$$
\left\{
\begin{aligned}
& d_t \overline{U} = \dfrac{D}{\Delta x^2} A_{N} \overline{U},\\
& \overline U(t_0) = (u_0(x_0),\ldots,u_0(x_N))^t,
\end{aligned}
\right.
$$

avec

$$
A_{N} = \frac{1}{(\Delta x)^2}\left( \begin{array}{cccccc}
- 2 & 2& 0 & 0  & 0  & 0  \\ 
1 & - 2& 1 & 0  & 0  & 0  \\
0 & 1 & - 2  & 1  & 0  & 0  \\
& & \ddots & \ddots & \ddots & \\
0 & 0& 0 & 1  & -2  & 1  \\ 
0 & 0& 0 & 0  & 2  & -2 \end{array} \right).
$$


Pour la simulation nous prenons $l=5$, $t_0=0.01$ et $D=1$. Le temps final est pris à $t_0+T$ avec $T=0.1$. Nous prenons aussi $1000$ points de maillage en espace, discrétisation qui restera fixée pour cette étude dans l'esprit EDO. On pourra consulter l'autre notebook pour l'analyse au sens des EDP où le pas en espace tend aussi vers zéro.

In [ ]:
#############################################
class heat_model:

    def __init__(self, d, xmin, xmax, nx) :
        self.d = d
        self.xmin = xmin
        self.xmax = xmax
        self.nx = nx
        self.dx = (xmax-xmin)/(nx-1)
        self.oneoverdxdx = 1/(self.dx*self.dx)
        
        doverdxdx = d/(self.dx**2)
        diag = np.repeat(-2, nx)
        sub_diag = np.repeat([1., 2.], [nx-2, 1])
        self.a = doverdxdx * diags([diag, sub_diag, np.flip(sub_diag)], [0, -1, 1])

    def fcn(self, t, y):
        return self.a.dot(y)
    
    def fcn_exact(self, t):
        xmin = self.xmin
        xmax = self.xmax
        d = self.d
        nx = self.nx
        dx = self.dx
        x = np.linspace(xmin, xmax, nx)
        y = (1/(2*np.sqrt(np.pi*t))) * np.exp(-(x*x)/(4.*t))
        return y

## Valeurs propres du Laplacien discret

En se basant sur l'analyse classique du spectre de la matrice $A$, on rappelle que les valeurs propres sont données par :

$$
\lambda_k = - \frac{4}{(\Delta x)^2}\,\sin^2\left( \frac{k\,\pi}{2(n+1)}\right),\quad k=0,\ldots,n-1.
$$


In [ ]:
xmin = -5.
xmax = 5.
lx = xmax-xmin
nx = 1000
dx = lx /(nx+1)

k = np.arange(nx)
    
disc_eig_vals = -(4/(dx*dx)) * np.sin((k*np.pi)/(2*(nx+1)))**2   

fig = go.Figure()

nx = np.arange(1000,10001,100)

for nxi in nx:
    dx = lx /(nxi+1)
    k = np.arange(nxi)
    disc_eig_vals = -(4/(dx*dx)) * np.sin((k*np.pi)/(2*(nxi+1)))**2
    fig.add_trace(go.Scatter(visible=False, x=k, y=disc_eig_vals, showlegend=True, name="Valeurs propres du Laplacien discrétisé"))

fig.data[0].visible = True
    
# Create and add slider
steps = []
for i, nxi in enumerate(nx):
    args = [{"visible": [(el==i) for el in range(len(fig.data))]}]
    step = dict(method="update", label = f"{nxi}", args=args)
    steps.append(step)
sliders = [dict(currentvalue={"prefix": "nx : "}, steps=steps)]

fig.update_layout(sliders=sliders, yaxis=dict(exponentformat='e'),  height=600,
                  legend=dict(x=0.65, y=0.99, bgcolor='rgba(0,0,0,0)'))
fig.show()

##  Euler explicite

In [ ]:
@dataclass
class ode_result:
    t: np.ndarray 
    y: np.ndarray    
    
#############################################
def forward_euler(tini, tend, nt, yini, fcn):
    
    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yn = np.copy(yini)

    for it, tn  in enumerate(t[:-1]):
        yn = yn + dt*fcn(tn, yn)
        
    return ode_result(t, yn)

In [ ]:
d = 1.
xmin = -5.
xmax = 5.
nx = 501
tini = 0.01
tend = 0.11

hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
dx = hm.dx
print('Dicrétisation spatiale : dx = ', dx)
x = np.linspace(xmin, xmax, nx)
fcn_exact = hm.fcn_exact
fcn = hm.fcn

yini = fcn_exact(tini)
yexa = fcn_exact(tend)

sol_qexa = solve_ivp(fcn, (tini, tend), yini, method='Radau', t_eval=[tend], rtol=1e-12, atol=1.e-12)
yqexa = sol_qexa.y[:,0]

fig = make_subplots(rows=2, cols=1, subplot_titles=("Solution", "Erreur"), vertical_spacing=0.15)

fig.add_trace(go.Scattergl(x=x, y=yexa, name=f'sol exacte', legendgroup='sol', line_color='rgb(76,114,176)'), row=1, col=1)
fig.add_trace(go.Scattergl(x=x, y=yqexa, name=f'sol quasi-exacte', mode='lines+markers', legendgroup='sol'), row=1, col=1)

nt_vn = int(2*(tend-tini)/(dx*dx)) + 1
nt = [nt_vn-20, nt_vn, nt_vn+4500, nt_vn+19500, nt_vn+39500, nt_vn+99500]
for nt_i in nt:
    #print('dt = ', (tend-tini)/(nt_i-1))
    sol_num = forward_euler(tini, tend, nt_i, yini, fcn)
    ysol = sol_num.y
    err_exa  = np.abs(yexa-ysol)
    err_qexa = np.abs(yqexa-ysol)
    err_dx   = np.abs(yqexa-yexa)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=ysol, name=f'sol numérique', mode='lines+markers', legendgroup='sol', line_color='rgb(85,168,104)'), row=1, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_exa, name='err calculée à partir <br>  de la solution exacte', mode='lines+markers', legendgroup='err', line_color='rgb(76,114,176)'), row=2, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_qexa, name='err calculée à partir <br>  de la solution quasi-exacte', mode='lines+markers', legendgroup='err', line_color='rgb(221,132,82)'), row=2, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_dx, name='err en dx', mode='lines+markers', legendgroup='err', line_color='rgb(196,78,82)'), row=2, col=1)

for i in range(2+4,6+4): fig.data[i].visible = True

steps = []
for i, nt_i in enumerate(nt):
    args = [{"visible": [(el==0) or (el==1) or (el==4*i+2) or (el==4*i+3) or (el==4*i+4) or (el==4*i+5)  for el in range(len(fig.data))]}]
    #print(args)
    step = dict(method="update", label = f"{nt_i}", args=args)
    steps.append(step)
sliders = [dict(active=1, currentvalue={"prefix": "nt : "}, steps=steps)]

legend = dict(x=0.8, bgcolor='rgba(0,0,0,0)', tracegroupgap=280, groupclick='toggleitem')
fig.update_layout(height=800, sliders=sliders, legend=legend)
fig.update_yaxes(exponentformat='e')
fig.show()